In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
app = FastAPI()
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
class model_input(BaseModel):
  text : str

In [ ]:
# loading the saved model
from google.colab import drive
drive.mount("/content/gdrive")
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_1 = BertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/Demo/BodyShaming")
model_1.to('cuda')
model_2 = BertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/Demo/Hatespeech")
model_2.to('cuda')
model_3 = BertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/Demo/Racism")
model_3.to('cuda')
model_4 = BertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/Demo/Sexism")
model_4.to('cuda')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
@app.post('/get_prediction')
def get_prediction(input_parameters : model_input):
  input_data = input_parameters.json()
  input_dictionary = json.loads(input_data)
  Text = input_dictionary['text']
  text = str(Text)
  inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
  outputs1 = model_1(**inputs)
  predictions1 = torch.nn.functional.softmax(outputs1.logits, dim=-1)
  predictions1 = predictions1.cpu().detach().numpy()
  outputs2 = model_2(**inputs)
  predictions2 = torch.nn.functional.softmax(outputs2.logits, dim=-1)
  predictions2 = predictions2.cpu().detach().numpy()
  outputs3 = model_3(**inputs)
  predictions3 = torch.nn.functional.softmax(outputs3.logits, dim=-1)
  predictions3 = predictions3.cpu().detach().numpy()
  outputs4 = model_4(**inputs)
  predictions4 = torch.nn.functional.softmax(outputs4.logits, dim=-1)
  predictions4 = predictions4.cpu().detach().numpy()
  X = {'not_bodyshaming' : str(predictions1[0][0]) ,
       'bodyshaming' : str(predictions1[0][1]) ,
       'not_hatespeech' : str(predictions2[0][0]) ,
       'hatespeech' : str(predictions2[0][1]) ,
       'not_racism' : str(predictions3[0][0]) ,
       'racism' : str(predictions3[0][1]) , 
       'not_sexism' : str(predictions4[0][0]) ,
       'sexism' : str(predictions4[0][1])
       }
  x  = json.dumps(X)     
  return x

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://1c40-34-124-128-64.ngrok.io


INFO:     Started server process [312]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2409:4043:2e0a:4944:1b5:3715:7002:77f0:0 - "POST /get_prediction HTTP/1.1" 200 OK
